In [15]:
import numpy as np
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

In [16]:
def teleportation_circuit(initial_state=[1/np.sqrt(2), 1/np.sqrt(2)]):
    """
    Creates a quantum teleportation circuit
    Args:
        initial_state (list): Initial state to teleport [alpha, beta]
    Returns:
        QuantumCircuit: Completed teleportation circuit
    """
    # Create quantum and classical registers
    q = QuantumRegister(3, 'q')    # 3 qubits: sender, auxiliary, receiver
    c = ClassicalRegister(2, 'c')   # 2 classical bits for measurements
    qc = QuantumCircuit(q, c)
    
    # Initialize qubit q0 to the state we want to teleport
    qc.initialize(initial_state, 0)
    qc.barrier()
    
    # Create Bell pair between q1 and q2
    qc.h(1)              # Apply Hadamard gate to q1
    qc.cx(1, 2)         # CNOT with q1 as control, q2 as target
    qc.barrier()
    
    # Apply teleportation protocol
    qc.cx(0, 1)         # CNOT between q0 and q1
    qc.h(0)             # Hadamard gate on q0
    qc.barrier()
    
    # Measure qubits q0 and q1
    qc.measure(0, 0)    # Measure q0 -> c0
    qc.measure(1, 1)    # Measure q1 -> c1
    qc.barrier()
    
    # Apply corrections on q2 based on measurements
    qc.x(2).c_if(c, 2)  # Apply X gate if c1=1
    qc.z(2).c_if(c, 1)  # Apply Z gate if c0=1
    
    return qc

In [17]:
def test_teleportation(initial_state=[1/np.sqrt(2), 1/np.sqrt(2)]):
    """
    Tests the teleportation protocol for a given initial state
    Args:
        initial_state (list): State to teleport [alpha, beta]
    """
    # Create and execute the circuit
    qc = teleportation_circuit(initial_state)
    
    # Use statevector simulator
    backend = Aer.get_backend('statevector_simulator')
    job = backend.run(qc)  # Using run() method directly from the backend
    result = job.result()
    
    print(f"Testing teleportation of state: {initial_state}")
    print("Circuit depth:", qc.depth())
    print("Number of gates:", len(qc.data))
    
    # Get final state
    statevector = result.get_statevector()
    print("\nFinal statevector:", statevector)
    
    # Display the circuit
    print("\nCircuit diagram:")
    print(qc.draw())
    
    return qc

In [18]:
# Test the protocol with different initial states
test_cases = [
    [1, 0],                    # |0⟩
    [0, 1],                    # |1⟩
    [1/np.sqrt(2), 1/np.sqrt(2)],  # |+⟩
    [1/np.sqrt(2), -1/np.sqrt(2)]  # |−⟩
]

In [19]:
# Run tests
for state in test_cases:
    print("\n" + "="*50)
    qc = test_teleportation(state)


Testing teleportation of state: [1, 0]
Circuit depth: 8
Number of gates: 13

Final statevector: Statevector([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))

Circuit diagram:
     ┌─────────────────┐ ░            ░      ┌───┐ ░ ┌─┐    ░               
q_0: ┤ Initialize(1,0) ├─░────────────░───■──┤ H ├─░─┤M├────░───────────────
     └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░               
q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░───────────────
                         ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░  ┌───┐  ┌───┐ 
q_2: ────────────────────░──────┤ X ├─░────────────░──╫──╫──░──┤ X ├──┤ Z ├─
                         ░      └───┘ ░            ░  ║  ║  ░  └─╥─┘  └─╥─┘ 
                                                      ║  ║    ┌──╨──┐┌──╨──┐
c: 2/═════════════════════════════════════════════════╩══╩════╡ 0x2 ╞╡ 0x1 ╞
                                                      0  1    └─────┘└───